
# Marching Cubes

Marching cubes is an algorithm to extract a 2D surface mesh from a 3D volume.
This can be conceptualized as a 3D generalization of isolines on topographical
or weather maps. It works by iterating across the volume, looking for regions
which cross the level of interest. If such regions are found, triangulations
are generated and added to an output mesh. The final result is a set of
vertices and a set of triangular faces.

The algorithm requires a data volume and an isosurface value. For example, in
CT imaging Hounsfield units of +700 to +3000 represent bone. So, one potential
input would be a reconstructed CT set of data and the value +700, to extract
a mesh for regions of bone or bone-like density.

This implementation also works correctly on anisotropic datasets, where the
voxel spacing is not equal for every spatial dimension, through use of the
`spacing` kwarg.


In [1]:
pip install pymeshlab

     |████████████████████████████████| 42.3MB 99kB/s 


In [2]:
pip install scikit-image

In [3]:
from tempfile import mkdtemp

import os.path as path

filename = path.join(mkdtemp(), 'newfile.dat')

In [4]:
import requests as req

res = req.get("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/n33w113/USGS_13_n33w113.tif")
from PIL import Image

from io import BytesIO 

imfil = BytesIO(res.content)
res = ""

In [5]:
im = Image.open(imfil)
imfil = ""

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (116899344 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [6]:
im.size

(10812, 10812)

In [7]:
import numpy as np

In [8]:
fp = np.memmap(filename, dtype='float32', mode='w+', shape=im.size)

In [9]:
fp[:] = np.array(im)

In [10]:
fp.flush()

In [11]:
im = ""

In [12]:
np.save("temp.npy",fp)

In [2]:

import numpy as np

In [3]:
from tempfile import mkdtemp

import os.path as path

filename = path.join(mkdtemp(), 'newfile.dat')

In [4]:
im = np.load("temp.npy")

In [5]:
fp = np.memmap(filename, dtype='float32', mode='w+', shape=im.shape)

In [6]:
fp[:] = im

In [7]:
del im

In [8]:
# this will be the point collection for all the verts
filename2 = path.join(mkdtemp(), 'newfile2.dat')

In [9]:
fp[:] = fp/2

In [10]:
fp.size

116899344

In [11]:
fp2 = np.memmap(filename2,dtype="float32",mode="w+",shape=(fp.size,3))

In [12]:
fp2[:,2] = fp.flatten()

In [13]:
## fill in the columns on the side 

chunks = 12
count_in_chunk = fp.size//chunks
for i in range(chunks):
    low = count_in_chunk*i
    high = count_in_chunk*(i+1)
    xs = np.remainder(np.arange(low,high),fp.shape[0])
    ys = np.floor_divide(np.arange(low,high),fp.shape[1])
    fp2[low:high,0] = ys
    fp2[low:high,1] = xs

In [15]:
fp2.flush()

In [16]:

fp2[:10,:]

memmap([[  0.     ,   0.     , 110.83905],
        [  0.     ,   1.     , 110.78907],
        [  0.     ,   2.     , 110.7499 ],
        [  0.     ,   3.     , 110.74183],
        [  0.     ,   4.     , 110.7258 ],
        [  0.     ,   5.     , 110.7009 ],
        [  0.     ,   6.     , 110.69189],
        [  0.     ,   7.     , 110.68074],
        [  0.     ,   8.     , 110.66818],
        [  0.     ,   9.     , 110.65174]], dtype=float32)

In [18]:
import pymeshlab 

In [20]:

filename3 = path.join(mkdtemp(), 'newfile3.dat')

In [23]:
#write into this, and we know thanks to point sampling that it will certainly be smaller than the number of rows
fp3 = np.memmap(filename3,dtype="float32",mode="w+",shape=(fp.size,3))


In [24]:
# zero things out so we can compare later
fp3[:] = np.zeros((fp.size,3))

In [27]:
chunks = 12
count_in_chunk = fp.size//chunks
assign_ind_start = 0
for i in range(chunks):
  print("starting",i)
  ms = pymeshlab.MeshSet()
  low = count_in_chunk*i
  high = count_in_chunk*(i+1)
  mesh = pymeshlab.Mesh(fp2[low:high,:])
  ms.add_mesh(mesh)
  ms.point_cloud_simplification(samplenum= 100000)
  sampled_verts = ms.current_mesh().vertex_matrix().astype("float32")
  fp3[assign_ind_start:assign_ind_start + sampled_verts.shape[0],:] = sampled_verts
  assign_ind_start += sampled_verts.shape[0]
  print("ending",i)

starting 0
ending 0
starting 1
ending 1
starting 2
ending 2
starting 3
ending 3
starting 4
ending 4
starting 5
ending 5
starting 6
ending 6
starting 7
ending 7
starting 8
ending 8
starting 9
ending 9
starting 10
ending 10
starting 11
ending 11


In [29]:
np.where(fp3 > 0)[0].shape

(4553843,)

In [ ]:
4553843


In [ ]:
ms = pymeshlab.MeshSet()
mesh = pymeshlab.Mesh(fp3)
ms.add_mesh(mesh)

In [ ]:
mesh = pymeshlab.Mesh(fp2)
ms.add_mesh(mesh)

In [1]:
import os 
os.listdir("/tmp")

['tmp69ocgu7l',
 'tmp3v1r6osw',
 'dap_multiplexer.INFO',
 'dap_multiplexer.65e5ad1548d0.root.log.INFO.20210524-224406.47',
 'tmpqx_0i9ox',
 'initgoogle_syslog_dir.0',
 'tmpvyetybic',
 'tmponrmg88w',
 'tmpsop2y6z3',
 'debugger_1mbmi251x6']

In [2]:
filename34553843

NameError: ignored

In [3]:
import glob

In [4]:
glob.glob("/tmp/**/*.dat")

['/tmp/tmp69ocgu7l/newfile2.dat',
 '/tmp/tmp3v1r6osw/newfile.dat',
 '/tmp/tmpqx_0i9ox/newfile3.dat',
 '/tmp/tmpvyetybic/newfile.dat',
 '/tmp/tmponrmg88w/newfile2.dat',
 '/tmp/tmpsop2y6z3/newfile.dat']

In [5]:
import numpy as np
fp3 = np.memmap("/tmp/tmpqx_0i9ox/newfile3.dat",dtype="float32",shape=(116899344,3),mode="r+")

In [14]:
test = np.random.random((4,2))
test

array([[0.38579132, 0.02490234],
       [0.21421561, 0.57601808],
       [0.8953352 , 0.47020028],
       [0.3951411 , 0.01151813]])

In [20]:
print(test> .5)
print(np.all(test>9,axis=1))

[[False False]
 [False  True]
 [ True False]
 [False False]]
[False False False False]


In [ ]:
np.all(fp3> 0,axis=0)

In [13]:
(fp3[:1000,:] > 0).shape

(1000, 3)

In [21]:
bool_mask = np.all(fp3 > 0,axis = 1)

In [23]:
fp3.shape[0]/fp3[bool_mask].shape[0]

77.06509169083343

In [24]:
recalculated_points = fp3[bool_mask]

In [25]:
recalculated_points.shape

(1516891, 3)

In [29]:
import os
from tempfile import mkdtemp
recalc_fname = os.path.join(mkdtemp(),"points.npy")

In [30]:
np.save(recalc_fname,recalculated_points)

In [32]:
import pymeshlab
ms = pymeshlab.MeshSet()
mesh = pymeshlab.Mesh(recalculated_points)
ms.add_mesh(mesh)

In [33]:
ms.compute_normals_for_point_sets()

In [34]:
ms.surface_reconstruction_screened_poisson()

In [35]:
ms.save_current_mesh("landscape_remeshed.ply")

In [36]:
from google.colab import files

In [37]:
files.download("landscape_remeshed.ply")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>